# Websocket with rosbridge
    This notebook is a tools for no ros computer to communicate with ros system through rosbridge.

In [ ]:
import roslibpy
import time
import threading

## global variable

In [ ]:
subscriber_dic = {}
thread = []
data = {}

# Connceting
Like socket connection, roslibpy connect to server by ip_number + port_number(default:9090)
Use .is_connect to know connect success or not

In [ ]:
host = input("input host IP: ")

client = roslibpy.Ros(host = host, port = 9090)
client.run()
print('Is ROS connected?', client.is_connected)

input host IP: 0.0.0.0
Is ROS connected? True


## Define subscriber function & subscriber call back

In [ ]:
def println(ros_list):
    if len(ros_list) == 0:
        print('Empty')
    else:
        for i in ros_list:
            print(i)
    return 0
   
# def publisher(host, target_topic, message_data):
#     print('this is publisher of ' + target_topic )
#     client = roslibpy.Ros(host = host, port = 9090)
#     talker = roslibpy.Topic(client, '/'+target_topic , 'std_msgs/String')
#     print('it will publish ' + message_data + ' to ' + target_topic)
#     talker.publish(roslibpy.Message({'data': message_data}))
#     print('Sending '+ message_data  +' ...')
#     talker.unadvertise()
    
def subscriber(host, topic_name, topic_type):
    #print('this is subscriber of ' + topic_name )
    client = roslibpy.Ros(host = host, port = 9090)
    listener = roslibpy.Topic(client, topic_name, topic_type, throttle_rate=1000)
    subscriber_dic[topic_name] = listener
    listener.subscribe(subscriber_callback)
    
def subscriber_callback(message):
    data[str(message['data'].split('@')[0])] = message['data'].split('@')[1]

In [ ]:
def show(datas):
    while True:
        print(datas, end='\r')
        time.sleep(0.1)

In [ ]:
show_data = threading.Thread(target = show, args=(data,))
show_data.start()

# Functional part
    

In [ ]:
while True:
    command = input('please input command(topic, node, subscriber, publisher, close, quit): ')
    if command == 'topic':
        print('topic')
        topic = client.get_topics()
        println(topic)
        
    elif command == 'node':
        print('node')
        node = client.get_nodes()
        println(node)
        
    elif command == 'service':
        print('service')
        service = client.get_services()
        println(service)
    
    elif command == 'para':
        print('parameter')
        para = client.get_params()
        println(para)
    elif command == 'publisher':
        
        raw_message = input('what message & topic you want to publish: topic,message ')
        target_topic = raw_message.split(',')[0]
        message_data = raw_message.split(',')[1]
        talker = roslibpy.Topic(client, '/'+target_topic , 'std_msgs/String')
        print('it will publish ' + message_data + ' to ' + target_topic)
        talker.publish(roslibpy.Message({'data': message_data}))
        print('Sending '+ message_data  +' ...')
        talker.unadvertise()
        #thread.append(threading.Thread(target = publisher, args=(host, target_topic, message_data)))
        #thread[len(thread)-1].start()
        
    elif command == 'subscriber':
        topic = client.get_topics()
        println(topic)
        print('')
        topic_name = input('what topic you want to listen: ')
        topic_name = '/' + topic_name
        topic_type = client.get_topic_type(topic_name)
        print('type_is ' + topic_type)
        
        if topic_name in subscriber_dic:
            pass
        else:
            thread.append(threading.Thread(target = subscriber, args=(host, topic_name, topic_type)))
            thread[len(thread)-1].start()

    elif command == 'close':
        topic = input('what topic you want to unsubscribe: ')
        try:
            subscriber_dic['/' + topic].unsubscribe()
            data.clear()
        except:
            print('invalid topic name')
    elif command == 'quit':
        client.terminate()
        break
    print('')    
    print('---------------------------------------------------')
    
    


please input command(topic, node, subscriber, publisher, close, quit): publisher
what message & topic you want to publish: topic,message string,testing publisher
it will publish testing publisher to string
Sending testing publisher ...

---------------------------------------------------
please input command(topic, node, subscriber, publisher, close, quit): subscriber
/rosout_agg
/rosout
/client_count
/connected_clients
/test
/test1
/string

what topic you want to listen: test
type_is std_msgs/String

---------------------------------------------------
please input command(topic, node, subscriber, publisher, close, quit): subscriber
/rosout_agghis is test message'}
/rosout
/client_count
/connected_clients
/test
/test1
/string

what topic you want to listen: test1
type_is std_msgs/Stringmessage'}

---------------------------------------------------
please input command(topic, node, subscriber, publisher, close, quit): close
what topic you want to unsubscribe: test: 'HI this is another t